In [1]:
import torch 
from torch import nn
import numpy as np
class ReviewClassifier(nn.Module):
    def __init__(self):
        super().__init__()        
        self.mlp = nn.Sequential(
            nn.Linear(384, 150),
            nn.ReLU(),
            nn.Linear(150,15),
            nn.ReLU(),
            nn.Linear(15, 1),
            nn.Sigmoid()
        ) 
        
    
    def forward(self, x):        
        return self.mlp(x)

In [2]:
state_dict = torch.load("review_classifier.bin")

In [3]:
import pandas as pd 
test_df = pd.read_feather("test_df.bin")

In [4]:
model = ReviewClassifier()
model.load_state_dict(state_dict)

<All keys matched successfully>

In [14]:
N = 48
test_example = torch.Tensor(np.array(test_df.embeddings[N]))
print(test_df.content[N])
print("Prawdopodobieństwo pozytywnego sentymentu: ",  round(model.forward(test_example).detach().numpy()[0], 3))


I am using this amplifier to power speakers in a training room. It gives good sound and is plenty loud enough for my needs.
Prawdopodobieństwo pozytywnego sentymentu:  0.925


In [15]:
from torchmetrics import Accuracy 

from tqdm import tqdm 
model.eval()
metric = Accuracy(task="binary")
for _, test_row in tqdm(test_df.iterrows(), desc='evaluating'):
    pred = model.forward(
        torch.Tensor(np.array(test_row.embeddings))
    )
    metric.update(pred, torch.Tensor([test_row.label]))
metric_value = metric.compute()

evaluating: 6000it [00:02, 2801.38it/s]


In [16]:
# ACCURACY ON TEST SET 
metric_value

tensor(0.8333)